
# Deploy FLAN-XXL Model with high performance on SageMaker using NVIDIA FasterTransformer

In this notebook, we explore how to host a FLAN-T5-XXL large language model with FP32 precision on SageMaker using the NVIDIA Faster Transformer. We use DJLServing as the model serving solution in this example. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent blog post (https://aws.amazon.com/blogs/machine-learning/deploy-bloom-176b-and-opt-30b-on-amazon-sagemaker-with-large-model-inference-deep-learning-containers-and-deepspeed/).


Model parallelism can help deploy large models that would normally be too large for a single GPU. With model parallelism, we partition and distribute a model across multiple GPUs. Each GPU holds a different part of the model, resolving the memory capacity issue for the largest deep learning models with billions of parameters. This notebook uses tensor parallelism techniques which allow GPUs to work simultaneously on the same layer of a model and achieve low latency inference relative to a pipeline parallel solution.

SageMaker has rolled out DeepSpeed container which now provides users with the ability to leverage the managed serving capabilities and help to provide the un-differentiated heavy lifting.

In this notebook, we deploy the open source [FLAN-T5-XXL](https://huggingface.co/google/flan-t5-xxl) model across GPUs on a ml.g5.12xlarge instance. 

The Flan-T5 are T5 models trained on the Flan collection of datasets which include: taskmaster2, djaym7/wiki_dialog, deepmind/code_contests, lambada, gsm8k, aqua_rat, esnli, quasc and qed.

[FasterTransformer](https://github.com/NVIDIA/FasterTransformer) implements a highly optimized transformer layer for both the encoder and decoder for inference. On Volta, Turing and Ampere GPUs, the computing power of Tensor Cores are used automatically when the precision of the data and weights are FP16.

FasterTransformer is built on top of CUDA, cuBLAS, cuBLASLt and C++. We provide at least one API of the following frameworks: TensorFlow, PyTorch and Triton backend. Users can integrate FasterTransformer into these frameworks directly. For supporting frameworks, we also provide example codes to demonstrate how to use, and show the performance on these frameworks


## Licence agreement
 - View license information https://huggingface.co/models?license=license:apache-2.0 before using the model.
 - This notebook is a sample notebook and not intended for production use. Please refer to the licence at https://github.com/aws/mit-0. 

 
 


In [1]:
!pip install sagemaker boto3 huggingface_hub --upgrade #--quiet

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 712.8/712.8 kB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 95.8 MB/s eta 0:00:0000:0100:01
  Created wheel for sagemaker: filename=sagemaker-2.144.0-py2.py3-none-any.whl size=958073 sha256=7f46dfeaf748fa5b6de2d1df916b24c448dbdd351d8c3c2fa66bf31834dcbed6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/f1/59/23/b56c0fd29b40391286e7a158100ef3633d6d7656efac1ce71e
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.106
    Uninstalling botocore-1.29.106:
      Successfully uninstalled botocore-1.29.106
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.106
    Uninstalling boto3-1.26.106:
      Successfully uninstalled bot

In [2]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

In [3]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = (
    "hf-large-model-djl/code_flan_xxl_ft"  # folder within bucket where code artifact will go
)

s3_model_prefix = (
    "hf-large-model-djl/model_flan_xxl_ft"  # folder within bucket where code artifact will go
)
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{model_bucket}/{s3_model_prefix}/"
print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

Pretrained model will be uploaded to ---- > s3://sagemaker-us-east-1-859489341279/hf-large-model-djl/model_flan_xxl_ft/


### Download the model from Hugging Face and upload the model artifacts on Amazon S3

In [4]:
from huggingface_hub import snapshot_download
from pathlib import Path
import os

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path(".")
local_model_path.mkdir(exist_ok=True)
model_name = "google/flan-t5-xxl"
# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
model_artifact = sess.upload_data(path=model_download_path, key_prefix=s3_model_prefix)
print(f"Model uploaded to --- > {model_artifact}")
print(f"We will set option.s3url={model_artifact}")

Model uploaded to --- > s3://sagemaker-us-east-1-859489341279/hf-large-model-djl/model_flan_xxl_ft
We will set option.s3url=s3://sagemaker-us-east-1-859489341279/hf-large-model-djl/model_flan_xxl_ft


In [6]:
!rm -rf {model_download_path}

## Create SageMaker compatible Model artifact,  upload Model to S3 and bring your own inference script.

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or postprocessing of the model's predictions.

However in this notebook, we demonstrate how to deploy a model with custom inference code.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - `serving.properties` and `model.py`.

The tarball is in the following format

```
code
├──── 
│   └── serving.properties
│   └── model.py
   

```

- `serving.properties` is the configuration file that can be used to configure the model server.
- `model.py` is the script handles any requests for serving.


#### Create serving.properties 

This is a configuration file to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration.

Here is a list of settings that we use in this configuration file -
- `engine`: The engine for DJL to use. In this case, it is **FasterTransformer**.
- `option.entryPoint`: The entrypoint python file or module. This should align with the engine that is being used. 
- `option.s3url`: Set this to the URI of the Amazon S3 bucket that contains the model. 

If you want to download the model from huggingface.co, you can set `option.modelid`. The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co. 
- `option.tensor_parallel_degree`: Set to the number of GPU devices over which DeepSpeed needs to partition the model. This parameter also controls the number of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests. For further reading on DeepSpeedyou can follow the link https://www.deepspeed.ai/tutorials/inference-tutorial/#initializing-for-inference. 

For more details on the configuration options and an exhaustive list, you can refer the documentation - https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html.


In [7]:
!mkdir -p code_fasterformer_flant5

In [8]:
%%writefile code_fasterformer_flant5/serving.properties
engine = FasterTransformer
option.tensor_parallel_degree = 4
option.s3url = {{s3url}}

Writing code_fasterformer_flant5/serving.properties


In [9]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("code_fasterformer_flant5/serving.properties").open().read())
Path("code_fasterformer_flant5/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize fformer/serving.properties | cat -n

     1	engine = FasterTransformer
     2	option.tensor_parallel_degree = 4
     3	option.s3url = s3://sagemaker-us-east-1-859489341279/hf-large-model-djl/model_flan_xxl_ft/


#### Create a model.py

In [10]:
%%writefile code_fasterformer_flant5/model.py
import fastertransformer as ft
from djl_python import Input, Output
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    T5Tokenizer,
    T5ForConditionalGeneration,
)
import os
import logging
import math
import torch


def load_model(properties):
    model_name = "google/flan-t5-xxl"
    tensor_parallel_degree = properties["tensor_parallel_degree"]
    pipeline_parallel_degree = 1
    model_location = properties["model_dir"]
    if "model_id" in properties:
        model_location = properties["model_id"]
    logging.info(f"Loading model in {model_location}")

    tokenizer = T5Tokenizer.from_pretrained(model_location)
    dtype = "fp32"
    model = ft.init_inference(
        model_location, tensor_parallel_degree, pipeline_parallel_degree, dtype
    )
    return model, tokenizer


def pipeline_generate(model, tokenizer, inputs):
    input_tokens = tokenizer.batch_encode_plus(inputs, return_tensors="pt", padding=True)
    batch_token, batch_seq_len = model.generate(input_tokens)

    decoded_batch_token = []
    for j in range(len(batch_token)):
        decoded_batch_token.append(
            tokenizer.decode(batch_token[j][0][: batch_seq_len[j][0]], skip_special_tokens=True)
        )

    return decoded_batch_token


model = None
tokenizer = None


def handle(inputs: Input):
    """
    inputs: Contains the configurations from serving.properties
    """
    global model, tokenizer

    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        return None

    data = inputs.get_as_json()

    input_sentences = data["inputs"]
    params = data["parameters"]

    outputs = model.pipeline_generate(input_sentences, **params)
    result = {"outputs": outputs}

    return Output().add_as_json(result)

Writing code_fasterformer_flant5/model.py


**Image URI for the DJL container is being used here**

In [11]:
inference_image_uri = image_uris.retrieve(
    framework="djl-fastertransformer", region=sess.boto_session.region_name, version="0.21.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-fastertransformer5.3.0-cu117


**Create the Tarball and then upload to S3 location**

In [12]:
!rm model.tar.gz
!tar czvf model.tar.gz code_fasterformer_flant5

code_fasterformer_flant5/
code_fasterformer_flant5/serving.properties
code_fasterformer_flant5/model.py


In [13]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-859489341279/hf-large-model-djl/code_flan_xxl_ft/model.tar.gz


### To create the end point the steps are:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.g5.24xlarge 
    
    b) ContainerStartupHealthCheckTimeoutInSeconds is 2400 to ensure health check starts after the model is ready    
3. Create the end point using the endpoint config created    
    

#### Create the Model
Use the image URI for the DJL container and the s3 location to which the tarball was uploaded.

The container downloads the model into the `/tmp` space on the instance because SageMaker maps the `/tmp` to the Amazon Elastic Block Store (Amazon EBS) volume that is mounted when we specify the endpoint creation parameter VolumeSizeInGB. It leverages `s5cmd`(https://github.com/peak/s5cmd) which offers a very fast download speed and hence extremely useful when downloading large models.

For instances like p4dn, which come pre-built with the volume instance, we can continue to leverage the `/tmp` on the container. The size of this mount is large enough to hold the model.


In [14]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"flan-xxl-fastertransformer")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

flan-xxl-fastertransformer-2023-04-06-03-43-35-040
Created Model: arn:aws:sagemaker:us-east-1:859489341279:model/flan-xxl-fastertransformer-2023-04-06-03-43-35-040


In [15]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            "ContainerStartupHealthCheckTimeoutInSeconds": 600,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:859489341279:endpoint-config/flan-xxl-fastertransformer-2023-04-06-03-43-35-040-config',
 'ResponseMetadata': {'RequestId': '3f1c70af-d7ff-4153-9b43-c3cb61440252',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3f1c70af-d7ff-4153-9b43-c3cb61440252',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '138',
   'date': 'Thu, 06 Apr 2023 03:43:36 GMT'},
  'RetryAttempts': 0}}

In [16]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-1:859489341279:endpoint/flan-xxl-fastertransformer-2023-04-06-03-43-35-040-endpoint


### This step can take ~ 10 min or longer so please be patient

In [17]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:859489341279:endpoint/flan-xxl-fastertransformer-2023-04-06-03-43-35-040-endpoint
Status: InService


#### While you wait for the endpoint to be created, you can read more about:
- [Deep Learning containers for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)
- [FasterTransformer](https://github.com/NVIDIA/FasterTransformer)

#### Leverage the Boto3 to invoke the endpoint. 

This is a generative model so we pass in a Text as a prompt and the Model will complete the sentence and return the results.

You can pass a batch of prompts as input to the model. This is done by setting `inputs` to the list of prompts. The model then returns a result for each prompt. The text generation can be configured using appropriate parameters. These `parameters` need to be passed to the endpoint as a dictionary of `kwargs`.

The below code sample illustrates the invocation of the endpoint using a prompts and also sets some parameters.

Here's a list of default arguments that's used by the model for inference. You can pass specific values based on the use case - 
```
default_args = dict(
            inputs_embeds=None,
            beam_width=1,
            max_seq_len=200,
            top_k=1,
            top_p=0.0,
            beam_search_diversity_rate=0.0,
            temperature=1.0,
            len_penalty=0.0,
            repetition_penalty=1.0,
            presence_penalty=None,
            min_length=0,
            random_seed=0,
            is_return_output_log_probs=False,
            is_return_cum_log_probs=False,
            is_return_cross_attentions=False,
            bad_words_list=None,
            stop_words_list=None
        )
```


In [18]:
%%time
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": ["Amazon.com is the best ", "Large model inference is"],
            "parameters": {"min_length": 20, "max_seq_len": 100, "temperature": 0.1},
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

CPU times: user 35.3 ms, sys: 0 ns, total: 35.3 ms
Wall time: 1.08 s


'{\n  "outputs":[\n    "Amazon.com is the best online shopping site. It is the best online shopping site because it provides the best customer service.",\n    "a method for estimating the parameters of a large model. The method is based on the assumption that the model is a linear combination of a set of smaller models."\n  ]\n}'

### Examples of prompt engineering for 'Zero Shot' NLP tasks
Here are some examples of how you can use the deployed model for zero-shot NLP tasks.

#### Common Sense reasoning

In [19]:
%%time
# -- Greedy generation
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": [
                "The world cup has kicked off in Los Angeles, United States.\n\nBased on the paragraph above can we conclude that \”The world cup takes place in United States.\”?\n\n[\”yes\”, \”no\”]"
            ],
            "parameters": {"temperature": 0.7},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

CPU times: user 3.41 ms, sys: 0 ns, total: 3.41 ms
Wall time: 83.5 ms


'{\n  "outputs":[\n    "yes"\n  ]\n}'

#### Text / Sentiment classification

In [20]:
%%time
# -- Greedy generation
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": [
                "Review:\nThis moive is so great and once again dazzles and delights us\nIs this movie review sentence negative or positive?\nOPTIONS:\n-positive \n-negative"
            ],
            "parameters": {},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

CPU times: user 2.99 ms, sys: 0 ns, total: 2.99 ms
Wall time: 75.7 ms


'{\n  "outputs":[\n    "positive"\n  ]\n}'

#### Translation

In [21]:
%%time
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": ["My name is Arthur\n\nTranslate to German"],
            "parameters": {"temperature": 0.7},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

CPU times: user 2.8 ms, sys: 0 ns, total: 2.8 ms
Wall time: 110 ms


'{\n  "outputs":[\n    "Mein Name ist Arthur"\n  ]\n}'

#### Article generation

In [22]:
%%time
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": [
                "Title: \”University has new facility coming up“\\nGiven the above title of an imaginary article, imagine the article.\n"
            ],
            "parameters": {"max_seq_len": 200, "temperature": 0.7},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

CPU times: user 40.1 ms, sys: 0 ns, total: 40.1 ms
Wall time: 3.08 s


'{\n  "outputs":[\n    "”University has new facility coming up“nThe University of California, Irvine, has announced that it will build a new facility for its School of Business Administration. The new building will be located on the Irvine campus, and will be the first new building constructed on the campus in more than a decade. The new building will be located on the southeast corner of the campus, near the intersection of University Drive and University Avenue. The new building will be a three-story, 60,000-square-foot building, with a total floor area of 70,000 square feet. The building will be constructed using a combination of steel and concrete, and will be designed to LEED (Leadership in Energy and Environmental Design) Silver Certification. The building will be constructed using a combination of steel and concrete, and will be designed to LEED (Leadership in Energy and Environmental Design) Silver Certification. The building will be constructed using a combination"\n  ]\n}'

## Clean Up

In [23]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '2e94a9f0-ab84-4d51-bf0f-a79dad907192',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2e94a9f0-ab84-4d51-bf0f-a79dad907192',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 06 Apr 2023 03:59:39 GMT'},
  'RetryAttempts': 0}}

In [24]:
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '45da230e-51fd-4c9f-8def-4701986e7fbf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '45da230e-51fd-4c9f-8def-4701986e7fbf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 06 Apr 2023 03:59:41 GMT'},
  'RetryAttempts': 0}}